In [1]:
# Import everything needed to edit/save/watch video clips
from moviepy.editor import VideoFileClip
from IPython.display import HTML

#import other pkgs
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import glob

%matplotlib qt
%matplotlib inline

from measurements import find_curvature, dist_center, annotate

In [2]:
#number of corners in x and y direction on chessboard used for calibration
nx = 9
ny = 6

#read calibration images
images_calib = glob.glob('./camera_cal/calibration*.jpg')

#corner coordinates of real undistorted chessboard
objpoints = []

#corner coordinates of the 2D chessboard
imgpoints = []

#generate xy coordinates for object points
objp = np.zeros((nx * ny, 3), np.float32)
objp[:,:2] = np.mgrid[0:nx, 0:ny].T.reshape(-1,2)

#read in images
for fname in images_calib:
    calib_img = mpimg.imread(fname)
    gray = cv2.cvtColor(calib_img, cv2.COLOR_RGB2GRAY)
#find corners    
    ret, corners = cv2.findChessboardCorners(gray, (nx, ny), None)
    
    if ret == True:
        imgpoints.append(corners)
        objpoints.append(objp)


In [3]:
#function to calibrate and undistort an image
def cal_undistort(img, imgpoints, objpoints, gray):
#calibrate camera
    ret, mtx, dist, rvecs, tvecs = cv2.calibrateCamera(objpoints, imgpoints, gray.shape[::-1], None, None)  
    undist = cv2.undistort(img, mtx, dist, None, mtx)
    return undist

In [4]:
#function to warp image
def warp(img):
    img_size = (img.shape[1], img.shape[0])
    corners = np.float32([[190,720],[589,457],[698,457],[1145,720]])
    new_top_left=np.array([corners[0,0],0])
    new_top_right=np.array([corners[3,0],0])
    offset=[150,0]
        
    src = np.float32([corners[0],corners[1],corners[2],corners[3]])
    dst = np.float32([corners[0]+offset,new_top_left+offset,new_top_right-offset ,corners[3]-offset])  
    
    M = cv2.getPerspectiveTransform(src, dst)
    Minv = cv2.getPerspectiveTransform(dst, src)
    warped = cv2.warpPerspective(img, M, img_size, flags = cv2.INTER_LINEAR)
    
    return warped, Minv

In [5]:
def binarize(img, s_thresh=(120, 255), sx_thresh=(20, 255),l_thresh=(40,255)):
    img = np.copy(img)
    
    # Convert to HLS color space
    hls = cv2.cvtColor(img, cv2.COLOR_RGB2HLS).astype(np.float)
    l_channel = hls[:,:,1]
    s_channel = hls[:,:,2]

    sobelx = cv2.Sobel(l_channel, cv2.CV_64F, 1, 0) 
    abs_sobelx = np.absolute(sobelx) 
    scaled_sobel = np.uint8(255*abs_sobelx/np.max(abs_sobelx))
    
    # Threshold sobelx
    sxbinary = np.zeros_like(scaled_sobel)
    sxbinary[(scaled_sobel >= sx_thresh[0]) & (scaled_sobel <= sx_thresh[1])] = 1
    
    # Threshold S channel
    s_binary = np.zeros_like(s_channel)
    s_binary[(s_channel >= s_thresh[0]) & (s_channel <= s_thresh[1])] = 1

    # Threshold L channel
    l_binary = np.zeros_like(l_channel)
    l_binary[(l_channel >= l_thresh[0]) & (l_channel <= l_thresh[1])] = 1
           
    binary = np.zeros_like(sxbinary)
    binary[((l_binary == 1) & (s_binary == 1) | (sxbinary==1))] = 1           
    return  binary

In [6]:
def find_lane_inds(img):
    #Histogram along all the columns in the lower half of the image
    histogram = np.sum(img[np.int(img.shape[0]/2):,:], axis = 0)
    #find peaks of the left and right halves of the histogram
    midpoint = np.int(histogram.shape[0]/2)
    leftx_base = np.argmax(histogram[:midpoint])
    rightx_base = np.argmax(histogram[midpoint:]) + midpoint
    
    #Window params
    nwindows = 9
    window_height = np.int(img.shape[0]/nwindows)
    margin = 100
    #min no.of px found to recenter window
    minpix = 50
    
    #Identify x and y pos of all non zero px in the image
    nonzero = img.nonzero()
    nonzeroy = np.array(nonzero[0])
    nonzerox = np.array(nonzero[1])

    #Set current pos of window
    leftx_current = leftx_base
    rightx_current = rightx_base

    #Create empty list to receive left and right lane pixel indices
    left_lane_inds = []
    right_lane_inds = []

    #Step through layers
    for window in range(nwindows):
        win_y_low = img.shape[0] - (window + 1) * window_height
        win_y_high = img.shape[0] - window * window_height

        #left lane window boundary
        win_xleft_low = leftx_current - margin
        win_xleft_high = leftx_current + margin

        #right lane window boundary
        win_xright_low = rightx_current - margin
        win_xright_high = rightx_current + margin
        
        #Identify non zero pixels in x and y within windows
        good_left_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
                          (nonzerox >= win_xleft_low) & (nonzerox < win_xleft_high)).nonzero()[0]
    
        good_right_inds = ((nonzeroy >= win_y_low) & (nonzeroy < win_y_high) & 
                          (nonzerox >= win_xright_low) & (nonzerox < win_xright_high)).nonzero()[0]
    
        #Append these indices to the list
        left_lane_inds.append(good_left_inds)
        right_lane_inds.append(good_right_inds)
    
        #recenter next window to the mean of indices found if pix > minpix
        if len(good_left_inds) > minpix:
            leftx_current = np.int(np.mean(nonzerox[good_left_inds]))
        if len(good_right_inds) > minpix:
            rightx_current = np.int(np.mean(nonzerox[good_right_inds]))
            
    return left_lane_inds, right_lane_inds

In [7]:
def dewarp(img, warped, Minv, left_fitx, right_fitx, ploty):
    # Recast the x and y points into usable format for cv2.fillPoly()
    pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
    pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
    pts = np.hstack((pts_left, pts_right))

    # Draw the lane onto the warped blank image
    cv2.fillPoly(warped, np.int_([pts]), (0,255, 0))

    # Warp the blank back to original image space using inverse perspective matrix (Minv)
    dewarped_img = cv2.warpPerspective(warped, Minv, (img.shape[1], img.shape[0]))
    return dewarped_img

In [8]:
# Define a class to receive the characteristics of each line detection
class Line():
    def __init__(self):
        # was the line detected in the last iteration?
        self.detected = False  
        # x values of the last n fits of the line
        self.recent_xfitted = [] 
        #average x values of the fitted line over the last n iterations
        self.bestx = None     
        #polynomial coefficients averaged over the last n iterations
        self.best_fit = None  
        #polynomial coefficients for the most recent fit
        self.current_fit = []  
        #radius of curvature of the line in some units
        self.radius_of_curvature = None 
        #distance in meters of vehicle center from the line
        self.line_base_pos = None 
        #difference in fit coefficients between last and new fits
        self.diffs = np.array([0,0,0], dtype='float') 
        #number of detected pixels
        self.px_count = None
    def add_fit(self, fit, inds):
        # add a found fit to the line, up to n
        if fit is not None:
            if self.best_fit is not None:
                # if we have a best fit, see how this new fit compares
                self.diffs = abs(fit-self.best_fit)
            if (self.diffs[0] > 0.001 or \
               self.diffs[1] > 1.0 or \
               self.diffs[2] > 100.) and \
               len(self.current_fit) > 0:
                # bad fit
                self.detected = False
            else:
                self.detected = True
                self.px_count = np.count_nonzero(inds)
                self.current_fit.append(fit)
                if len(self.current_fit) > 5:
                    # throw out old fits, keep newest n
                    self.current_fit = self.current_fit[len(self.current_fit)-5:]
                self.best_fit = np.average(self.current_fit, axis=0)
        # or remove one from the history, if not found
        else:
            self.detected = False
            if len(self.current_fit) > 0:
                # throw out oldest fit
                self.current_fit = self.current_fit[:len(self.current_fit)-1]
            if len(self.current_fit) > 0:
                # if there are still any fits in the queue, best_fit is their average
                self.best_fit = np.average(self.current_fit, axis=0)


In [16]:
def lane_find(img):
    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
#step1: undistort
    undist = cal_undistort(img, imgpoints, objpoints, gray)
    
#step2: warp    
    warped, Minv = warp(undist)
    
#step3: threshodling 
    binary_img = np.zeros_like(gray)
    margin = 100    
    binary_img = binarize(warped)
    
#step4.1: for frame0, compute histogram to find left and right lane starting position,
#######   and use sliding window search to find left and right lane pixel indices
    #if both left and right lanes are not detected use sliding window search
    if not l_lane.detected or not r_lane.detected:
        left_lane_inds, right_lane_inds = find_lane_inds(binary_img)

        #concatenate the array of indices
        left_lane_inds = np.concatenate(left_lane_inds)
        right_lane_inds = np.concatenate(right_lane_inds)

        #extract the left and right lane pixel pos
        nonzero = binary_img.nonzero()
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        leftx = nonzerox[left_lane_inds]
        lefty = nonzeroy[left_lane_inds]

        rightx = nonzerox[right_lane_inds]
        righty = nonzeroy[right_lane_inds]

        #fit a second order polynomial - gives the coefficients
        left_fit = np.polyfit(lefty, leftx, 2)
        right_fit = np.polyfit(righty, rightx, 2)

#step4.2: for frame>0, 
#Identify x and y pos of all non zero px in the image
    else:
        left_fit = l_lane.best_fit
        right_fit = r_lane.best_fit
        
        nonzero = binary_img.nonzero()
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])

        #get left and right lane indices within margin of the polyfit
        left_lane_inds = ((nonzerox > (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] - margin)) & 
                          (nonzerox < (left_fit[0]*(nonzeroy**2) + left_fit[1]*nonzeroy + left_fit[2] + margin))) 

        right_lane_inds = ((nonzerox > (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] - margin)) & 
                           (nonzerox < (right_fit[0]*(nonzeroy**2) + right_fit[1]*nonzeroy + right_fit[2] + margin)))  

        # Again, extract left and right line pixel positions
        leftx = nonzerox[left_lane_inds]
        lefty = nonzeroy[left_lane_inds] 
        rightx = nonzerox[right_lane_inds]
        righty = nonzeroy[right_lane_inds]

        # Fit a second order polynomial to each
        left_fit = np.polyfit(lefty, leftx, 2)
        right_fit = np.polyfit(righty, rightx, 2) 
        
    l_lane.add_fit(left_fit, left_lane_inds)
    r_lane.add_fit(right_fit, right_lane_inds)

#step5: draw the current best fit if it exists
    if l_lane.best_fit is not None and r_lane.best_fit is not None:
        #Generate x and y values for plotting
        ploty = np.linspace(0, binary_img.shape[0]-1, binary_img.shape[0] )
        left_fitx = left_fit[0]*ploty**2 + left_fit[1]*ploty + left_fit[2]
        right_fitx = right_fit[0]*ploty**2 + right_fit[1]*ploty + right_fit[2]
    
        #Create an output image to draw the result
        #window_img = np.dstack((binary_img, binary_img, binary_img))*255
        window_img = np.zeros_like(img)
        # Recast the x and y points into usable format for cv2.fillPoly()
        pts_left = np.array([np.transpose(np.vstack([left_fitx, ploty]))])
        pts_right = np.array([np.flipud(np.transpose(np.vstack([right_fitx, ploty])))])
        pts = np.hstack((pts_left, pts_right))
 
        cv2.polylines(window_img, np.int32([pts_left]), isClosed=False, color=(255,0,255), thickness=35)
        cv2.polylines(window_img, np.int32([pts_right]), isClosed=False, color=(0,255,255), thickness=35)

        bin_img_fit = np.copy(window_img)
        #dewarp the window_img, merge with undistorted image
        dewarped_img = dewarp(img, window_img, Minv, left_fitx, right_fitx, ploty)
        overlay = cv2.addWeighted(undist, 1, dewarped_img, 0.3, 0)

        #measure radius of curvature, distance from center of road
        left_curverad, right_curverad, rad = find_curvature(ploty, leftx, rightx, lefty, righty)
        d_center = dist_center(overlay, left_fit, right_fit, binary_img)
    
        #annotate frame
        annotate_img = annotate(overlay, rad, d_center)
    else:
        annotate_img = img;
        
#step6: diagnostics window
    diagnostic_output = False
    if diagnostic_output:
        # put together multi-view output
        diag_img = np.zeros((720,1280,3), dtype=np.uint8)
        
        # original output (top left)
        diag_img[0:360,0:640,:] = cv2.resize(annotate_img,(640,360))
        
        # binary overhead view (top right)
        binary_img = np.dstack((binary_img*255, binary_img*255, binary_img*255))
        resized_bin_img = cv2.resize(binary_img,(640,360))
        diag_img[0:360,640:1280, :] = resized_bin_img

        bin_img_fit = cv2.resize(bin_img_fit, (640,360))
        diag_img[360:720,640:1280,:] = bin_img_fit
        
        # diagnostic data (bottom left)
        color_ok = (200,255,155)
        color_bad = (255,155,155)
        font = cv2.FONT_HERSHEY_DUPLEX
        if left_fit is not None:
            text = 'This fit L: ' + ' {:0.6f}'.format(left_fit[0]) + \
                                    ' {:0.6f}'.format(left_fit[1]) + \
                                    ' {:0.6f}'.format(left_fit[2])
        else:
            text = 'This fit L: None'
        cv2.putText(diag_img, text, (40,380), font, .5, color_ok, 1, cv2.LINE_AA)
        
        if right_fit is not None:
            text = 'This fit R: ' + ' {:0.6f}'.format(right_fit[0]) + \
                                    ' {:0.6f}'.format(right_fit[1]) + \
                                    ' {:0.6f}'.format(right_fit[2])
        else:
            text = 'This fit R: None'
        cv2.putText(diag_img, text, (40,400), font, .5, color_ok, 1, cv2.LINE_AA)
        
        text = 'Best fit L: ' + ' {:0.6f}'.format(l_lane.best_fit[0]) + \
                                ' {:0.6f}'.format(l_lane.best_fit[1]) + \
                                ' {:0.6f}'.format(l_lane.best_fit[2])
        cv2.putText(diag_img, text, (40,440), font, .5, color_ok, 1, cv2.LINE_AA)
        
        text = 'Best fit R: ' + ' {:0.6f}'.format(r_lane.best_fit[0]) + \
                                ' {:0.6f}'.format(r_lane.best_fit[1]) + \
                                ' {:0.6f}'.format(r_lane.best_fit[2])
        cv2.putText(diag_img, text, (40,460), font, .5, color_ok, 1, cv2.LINE_AA)
        
        text = 'Diffs L: ' + ' {:0.6f}'.format(l_lane.diffs[0]) + \
                             ' {:0.6f}'.format(l_lane.diffs[1]) + \
                             ' {:0.6f}'.format(l_lane.diffs[2])
        if l_lane.diffs[0] > 0.001 or \
           l_lane.diffs[1] > 1.0 or \
            l_lane.diffs[2] > 100. :
            diffs_color = color_bad
        else:
            diffs_color = color_ok
        cv2.putText(diag_img, text, (40,500), font, .5, diffs_color, 1, cv2.LINE_AA)
        
        text = 'Diffs R: ' + ' {:0.6f}'.format(r_lane.diffs[0]) + \
                             ' {:0.6f}'.format(r_lane.diffs[1]) + \
                             ' {:0.6f}'.format(r_lane.diffs[2])
        if r_lane.diffs[0] > 0.001 or \
           r_lane.diffs[1] > 1.0 or \
            r_lane.diffs[2] > 100.:
            diffs_color = color_bad
        else:
            diffs_color = color_ok
        cv2.putText(diag_img, text, (40,520), font, .5, diffs_color, 1, cv2.LINE_AA)
        
        text = 'Good fit count L:' + str(len(l_lane.current_fit))
        cv2.putText(diag_img, text, (40,560), font, .5, color_ok, 1, cv2.LINE_AA)
        
        text = 'Good fit count R:' + str(len(r_lane.current_fit))
        cv2.putText(diag_img, text, (40,580), font, .5, color_ok, 1, cv2.LINE_AA)
        
        annotate_img = diag_img        
        
    return annotate_img  

In [17]:
l_lane = Line()
r_lane = Line()
#video processing
clip_output = './challenge_video_output.mp4'
## To speed up the testing process you may want to try your pipeline on a shorter subclip of the video
## To do so add .subclip(start_second,end_second) to the end of the line below
## Where start_second and end_second are integer values representing the start and end of the subclip
## You may also uncomment the following line for a subclip of the first 5 seconds
#clip = VideoFileClip("challenge_video.mp4").subclip(0,5)

clip = VideoFileClip("challenge_video.mp4")
process_clip = clip.fl_image(lane_find) #NOTE: this function expects color images!!
%time process_clip.write_videofile(clip_output, audio=False)

[MoviePy] >>>> Building video ./challenge_video_output.mp4
[MoviePy] Writing video ./challenge_video_output.mp4


100%|██████████| 485/485 [17:14<00:00,  2.12s/it]


[MoviePy] Done.
[MoviePy] >>>> Video ready: ./challenge_video_output.mp4 

CPU times: user 14min 18s, sys: 9.06 s, total: 14min 27s
Wall time: 17min 20s


In [18]:
HTML("""
<video width="960" height="540" controls>
  <source src="{0}">
</video>
""".format(clip_output))